In [1]:
import pandas as pd

In [2]:
import sys
sys.path.append('../')

import seaborn as sns
from data_model import Individual
from sys_utils import load_model

checkpoint_path = '../checkpoints_dev'

individuals = load_model(
        Individual, name=checkpoint_path + "/individuals.jsonl"
    )

from tqdm import tqdm
template = 'simple_white'

100%|████████████████████████████████| 225636/225636 [00:11<00:00, 19435.76it/s]


In [62]:
individuals_filtered = [x for x in individuals if x.regions != None]

df_individuals = [
    {
        "wikidata_id": x.id.wikidata_id,
        "region_code": x.regions,
        "cultural_score":x.cultural_score,
         "year": x.impact_years,
        'birthyear':x.id.birthyear
    }
    for x in individuals_filtered
]


from data_model_region import Region
df = pd.DataFrame(df_individuals)
df = df.explode('region_code')

regions = load_model(
    Region, name=checkpoint_path + "/regions.jsonl"
)

df_regions = [
        {
            "region_code": x.code,
            "region_name": x.name,
        }
        for x in regions
    ]

df_regions = pd.DataFrame(df_regions)
df = pd.merge(df, df_regions, on = 'region_code')
df = df.dropna()


In [79]:
import numpy as np

In [97]:
df['decade'] = df['birthyear'].apply(lambda x: round(x / 50) * 50)
# Group by region_name and sort each group by cultural_score in descending order
groups = df.groupby('region_name')
sorted_groups = [g.sort_values('cultural_score', ascending=False).reset_index(drop=True) for _, g in groups]

# Filter out the top 20% of wikidata_id for each region_name
filtered_groups = [g.iloc[int(len(g)*0.1):] for g in sorted_groups]


# Concatenate the filtered groups back into a single dataframe
df_less_top = pd.concat(filtered_groups).reset_index(drop=True)
df_fig = df_less_top.groupby(['region_name', 'decade'])['cultural_score'].sum().reset_index()
df_fig['cultural_score'] = np.log(1 + df_fig['cultural_score'])


In [101]:
region = 'Chinese world'
min_date = -800
max_date = 1850

df_fig_filtered = df_fig[(df_fig['region_name']==region)&(df_fig['decade']>=min_date)&(df_fig['decade']<=max_date)]
df_fig_filtered['cultural_score'] = (df_fig_filtered['cultural_score'] - df_fig_filtered['cultural_score'].min()) / (df_fig_filtered['cultural_score'].max() - df_fig_filtered['cultural_score'].min())


import plotly.express as px
fig = px.line(df_fig_filtered, 
              x = 'decade', 
              y = 'cultural_score', 
              width = 1000, 
              height = 600, 
              title = f'{region} (Top 90% Individuals)',
              template = 'plotly_white')

fig.update_xaxes(showgrid=True, showticklabels=True, zeroline=False)
fig.update_yaxes(showgrid=True, showticklabels=True, zeroline=False)
fig.update_layout(title_font=dict(size=20))

fig.update_layout(
    xaxis_title="",
    yaxis_title='Log Cultural Index')
import plotly
plotly.offline.plot(fig)

/var/folders/lx/9b13r6ln63jb54_3xlf9b_380000gn/T/ipykernel_33702/2604702602.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



'temp-plot.html'

'temp-plot.html'